# Advanced EDA: Long to Wide
**Dr. Dave Wanik - University of Connecticut**

-------------------------------------------------

Here is an exercise of how to reshape and aggregate data - oftentimes data might be presented in 'long' form like a transacational database (looks like time series data), and it might be easier to analyze if the data was reshaped.

We will leverage data from ctdata.org - a fantastic website for all things related to Connecticut data! It's smaller to work with than Census data and is local, so it's great for demonstration purposes.

In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
# load some packages
import pandas as pd
import numpy as np

You can access these files from here:
* [median-home-value-town-2017.csv](https://drive.google.com/file/d/1a8H1J_oTZozny5g359bFKALy1JbJ4w34/view?usp=sharing)
* [educationalattainment2017.csv](https://drive.google.com/file/d/19_B_CBxnhlHItMDaPcVo2X8LGHH03JcH/view?usp=sharing)

In [126]:
 # import some data (we'll use median house price data and educational attainment data)
df_home = pd.read_csv("/content/drive/MyDrive/Online Materials/Module 2: Dirty Data and Parametric Models/Week 1: Advanced EDA/data/median-home-value-town-2017.csv")
df_ed = pd.read_csv('/content/drive/MyDrive/Online Materials/Module 2: Dirty Data and Parametric Models/Week 1: Advanced EDA/data/educationalattainment2017.csv')


# Exploring housing data
If you didn't know, Connecticut has 169 towns. Let's use this fact to help us appreciate the data that we are exploring.

In [127]:
df_home.shape #5440 rows and 6 columns... but CT only has 169 towns! what's going on?

(5440, 6)

In [128]:
# show head
df_home.head()

,Town,FIPS,Year,Measure Type,Variable,Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.00
1,Andover,901301080,2006-2010,Number,Margins of Error,21727.00
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,901301080,2006-2010,Ratio to State Median,Margins of Error,0.07
4,Andover,901301080,2007-2011,Number,Median Home Value,303500.00


In [129]:
# data types
df_home.dtypes

Town             object
FIPS              int64
Year             object
Measure Type     object
Variable         object
Value           float64
dtype: object

In [130]:
# unique values in each column
# link: https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/
# btw, look at his cookbook and other flashcard materials

# two ways of doing this
df_home.Year.unique()

array(['2006-2010', '2007-2011', '2008-2012', '2009-2013', '2010-2014',
       '2011-2015', '2012-2016', '2013-2017'], dtype=object)

In [131]:
df_home["Year"].nunique()

8

OK - so there are 8 years of data... perhaps that's where the 5440 rows are coming from. Let's use code to check.

In [132]:
5440/8/4 # YES! That must be it.

170.0

# Unique values in a column
In addition to using our background knowledge that Connecticut has 169 towns, perhaps there's another way we can do this...

Let's try a few different functions from pandas
* `unique()` Hash table-based unique. Uniques are returned in order of appearance. This does NOT sort.
Significantly faster than numpy.unique. Includes NA values.
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html
* `nunique()`. Count distinct observations over requested axis. Return Series with number of distinct observations. Can ignore NaN values.
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html
* `value_counts()` Return a Series containing counts of unique values. The resulting object will be in descending order so that the first element is the most frequently-occurring element. Excludes NA values by default.
  * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

`unique` gives you a list of all of the unique values in a column.

In [133]:
# look at unique towns

# unique(), nunique(), value_counts()
df_home.Town.unique() # this gives a list of the towns
# df_home['Town'].unique() 

array(['Andover', 'Ansonia', 'Ashford', 'Avon', 'Barkhamsted',
       'Beacon Falls', 'Berlin', 'Bethany', 'Bethel', 'Bethlehem',
       'Bloomfield', 'Bolton', 'Bozrah', 'Branford', 'Bridgeport',
       'Bridgewater', 'Bristol', 'Brookfield', 'Brooklyn', 'Burlington',
       'Canaan', 'Canterbury', 'Canton', 'Chaplin', 'Cheshire', 'Chester',
       'Clinton', 'Colchester', 'Colebrook', 'Columbia', 'Connecticut',
       'Cornwall', 'Coventry', 'Cromwell', 'Danbury', 'Darien',
       'Deep River', 'Derby', 'Durham', 'East Granby', 'East Haddam',
       'East Hampton', 'East Hartford', 'East Haven', 'East Lyme',
       'East Windsor', 'Eastford', 'Easton', 'Ellington', 'Enfield',
       'Essex', 'Fairfield', 'Farmington', 'Franklin', 'Glastonbury',
       'Goshen', 'Granby', 'Greenwich', 'Griswold', 'Groton', 'Guilford',
       'Haddam', 'Hamden', 'Hampton', 'Hartford', 'Hartland', 'Harwinton',
       'Hebron', 'Kent', 'Killingly', 'Killingworth', 'Lebanon',
       'Ledyard', 'Lisbon', '

In [134]:
df_home['Town'].nunique() # this gives the count

170

Wait a second... ***what's going on?!***

There is STATE TOTAL data hiding in the dataset - can you see it above? Perhaps you will see it with `value_counts()`. Always try to use your domain knowledge to help you find deeper, more meaningful patterns.

In [135]:
x = df_home['Town'].value_counts() # this gives count of rows PER unique value
x # 32 observations per town... why do we think this is?

Andover       32
Putnam        32
Oxford        32
Plainfield    32
Plainville    32
              ..
Griswold      32
Groton        32
Guilford      32
Haddam        32
Woodstock     32
Name: Town, Length: 170, dtype: int64

Let's look at all the variables were are measuring.

In [136]:
x = df_home['Variable'].value_counts()
x # two variables

Median Home Value    2720
Margins of Error     2720
Name: Variable, dtype: int64

In [137]:
x = df_home['Measure Type'].value_counts()
x # two variables

Number                   2720
Ratio to State Median    2720
Name: Measure Type, dtype: int64

SO! We have 8 years of data and 4 measurements per year per town. This is why it comes up as 32 rows per town.

In [138]:
# here's the other way
print(df_home['Year'].nunique()) # 8 years
print(df_home['Year'].unique()) # the list of the 8 years

8
['2006-2010' '2007-2011' '2008-2012' '2009-2013' '2010-2014' '2011-2015'
 '2012-2016' '2013-2017']


# Selecting columns
Let's just select the data that we think might be most useful. 

There are a few ways you can do this, but I'll just make a list of the variables that I think are most useful.

In [139]:
# let's subset the Town, Year, Variable and Value columns
df_home_subset = df_home[['Town', 'Year', 'Measure Type', 'Variable', 'Value']]
print(df_home_subset.shape) # we dropped a column
df_home_subset.head()

(5440, 5)


,Town,Year,Measure Type,Variable,Value
0,Andover,2006-2010,Number,Median Home Value,276300.00
1,Andover,2006-2010,Number,Margins of Error,21727.00
2,Andover,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,2006-2010,Ratio to State Median,Margins of Error,0.07
4,Andover,2007-2011,Number,Median Home Value,303500.00


`.iloc` is also useful for subsetting rows and columns (you can use a numeric index instead of names). 

In [140]:
# we could also have used .iloc to select columns
# this is a really nice tutorial on subsetting rows and columns
# link: https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

df_home_subset = df_home.iloc[:,[0,2,3,4,5]] #df.iloc[[rows],[columns]]
print(df_home_subset.shape)
df_home_subset.head()

(5440, 5)


,Town,Year,Measure Type,Variable,Value
0,Andover,2006-2010,Number,Median Home Value,276300.00
1,Andover,2006-2010,Number,Margins of Error,21727.00
2,Andover,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,2006-2010,Ratio to State Median,Margins of Error,0.07
4,Andover,2007-2011,Number,Median Home Value,303500.00


While `.iloc` requires index numbers, `.loc` is useful for column names. Sometimes it is helpful to define a list of the column names and keeping this as a separate variable outside.

In [141]:
names = ['Town', 'Year', 'Measure Type', 'Variable', 'Value']
df_home_subset = df_home.loc[:,names] #df.loc[[rows],[columns]]
print(df_home_subset.shape)
df_home_subset.head()

(5440, 5)


,Town,Year,Measure Type,Variable,Value
0,Andover,2006-2010,Number,Median Home Value,276300.00
1,Andover,2006-2010,Number,Margins of Error,21727.00
2,Andover,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,2006-2010,Ratio to State Median,Margins of Error,0.07
4,Andover,2007-2011,Number,Median Home Value,303500.00


# Selecting rows
To keep things simple, let's just analyze one year of data (`2006-2010`).

In [142]:
# now let's subset rows that Year = 2006-2010
df_home_2006 = df_home[df_home['Year']=="2006-2010"]
print(df_home_2006.shape)
df_home_2006.head()

(680, 6)


,Town,FIPS,Year,Measure Type,Variable,Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.00
1,Andover,901301080,2006-2010,Number,Margins of Error,21727.00
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,901301080,2006-2010,Ratio to State Median,Margins of Error,0.07
32,Ansonia,900901220,2006-2010,Number,Median Home Value,267300.00


In [143]:
# select a few rows where town is equal to Connecticut
df_home_2006[df_home_2006['Town']=='Connecticut']

,Town,FIPS,Year,Measure Type,Variable,Value
960,Connecticut,9,2006-2010,Number,Median Home Value,296500.00
961,Connecticut,9,2006-2010,Number,Margins of Error,1092.00
962,Connecticut,9,2006-2010,Ratio to State Median,Median Home Value,1.00
963,Connecticut,9,2006-2010,Ratio to State Median,Margins of Error,0.01


We see that there are 680 rows... remember... there is a Connecticut total hiding in there! Let's delete that now.

In [144]:
# delete a row where true
df_home_2006 = df_home_2006[df_home_2006['Town']!='Connecticut'] #!= means 'is not'
print(df_home_2006.shape)

(676, 6)


In [145]:
676/169 # perfect!

4.0

In [146]:
# how can we check that it worked? return unique values. it worked!
df_home_2006.Year.unique() # don't forget the ()
# df_home['Year'].unique()

array(['2006-2010'], dtype=object)

## Pt 1: Median Home Value
Let's subset rows for median home value and go from long to wide. Later, we will do the same for ratio to state mean data and then join the two together.

In [147]:
# can we reshape the data, so that it goes to wide format instead of long format?
tmp1 = df_home_2006[df_home_2006['Variable']=="Median Home Value"]
print(tmp1.shape)
tmp1.head()

(338, 6)


,Town,FIPS,Year,Measure Type,Variable,Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.00
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
32,Ansonia,900901220,2006-2010,Number,Median Home Value,267300.00
34,Ansonia,900901220,2006-2010,Ratio to State Median,Median Home Value,0.90
64,Ashford,901501430,2006-2010,Number,Median Home Value,267900.00


In [148]:
# still have two entires in "Measure Type", let's make two more tmpdf's
# this got rid of the margin of error in the original data
tmp2 = tmp1[tmp1["Measure Type"]=="Number"]
print(tmp2.shape)
tmp2.head()

(169, 6)


,Town,FIPS,Year,Measure Type,Variable,Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.0
32,Ansonia,900901220,2006-2010,Number,Median Home Value,267300.0
64,Ashford,901501430,2006-2010,Number,Median Home Value,267900.0
96,Avon,900302060,2006-2010,Number,Median Home Value,421100.0
128,Barkhamsted,900502760,2006-2010,Number,Median Home Value,294900.0


In [149]:
# rename the value colum to "Median Home Value"
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
tmp2.rename(columns={"Value": "Median Home Value"}, inplace=True)
tmp2.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Town,FIPS,Year,Measure Type,Variable,Median Home Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.0
32,Ansonia,900901220,2006-2010,Number,Median Home Value,267300.0
64,Ashford,901501430,2006-2010,Number,Median Home Value,267900.0
96,Avon,900302060,2006-2010,Number,Median Home Value,421100.0
128,Barkhamsted,900502760,2006-2010,Number,Median Home Value,294900.0


In [150]:
# what's up with that SettingWithCopyWarning? Check this out for more.
# link: https://www.dataquest.io/blog/settingwithcopywarning/

In [151]:
# drop a column (there are many ways)
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
tmp2.drop(['Measure Type', 'Variable'], axis=1, inplace=True)
tmp2.head() # now this looks nice! 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Town,FIPS,Year,Median Home Value
0,Andover,901301080,2006-2010,276300.0
32,Ansonia,900901220,2006-2010,267300.0
64,Ashford,901501430,2006-2010,267900.0
96,Avon,900302060,2006-2010,421100.0
128,Barkhamsted,900502760,2006-2010,294900.0


In [152]:
tmp2.shape

(169, 4)

## Pt. 2: Ratio to State Median
Repeat all steps but for a different variable ('Ratio to State Median')

In [153]:
# we can also repeat these steps for "Ratio to State Median", then we can join everything together
tmp3 = tmp1[tmp1['Measure Type']=='Ratio to State Median']
print(tmp3.shape)
tmp3.head()

(169, 6)


,Town,FIPS,Year,Measure Type,Variable,Value
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
34,Ansonia,900901220,2006-2010,Ratio to State Median,Median Home Value,0.90
66,Ashford,901501430,2006-2010,Ratio to State Median,Median Home Value,0.90
98,Avon,900302060,2006-2010,Ratio to State Median,Median Home Value,1.42
130,Barkhamsted,900502760,2006-2010,Ratio to State Median,Median Home Value,0.99


In [154]:
tmp3.describe()

,FIPS,Value
count,1.690000e+02,169.000000
mean,9.007641e+08,1.165740
std,4.415947e+05,0.543892
min,9.001047e+08,0.560000
25%,9.003601e+08,0.860000
50%,9.007225e+08,1.000000
75%,9.011424e+08,1.240000
max,9.015882e+08,3.370000


In [155]:
# rename the column "Value" to "Ratio to State Median"
tmp3.rename(columns={"Value": "Ratio to State Median"}, inplace=True)
tmp3.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Town,FIPS,Year,Measure Type,Variable,Ratio to State Median
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
34,Ansonia,900901220,2006-2010,Ratio to State Median,Median Home Value,0.90
66,Ashford,901501430,2006-2010,Ratio to State Median,Median Home Value,0.90
98,Avon,900302060,2006-2010,Ratio to State Median,Median Home Value,1.42
130,Barkhamsted,900502760,2006-2010,Ratio to State Median,Median Home Value,0.99


In [156]:
tmp3.drop(['Measure Type', 'Variable'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [157]:
print(tmp2.shape, tmp3.shape) # Median home value = tmp2, ratio to state median = tmp3

(169, 4) (169, 4)


In [158]:
# now we can do a join, not on index, but by columns
# link: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
# df1.merge(df2, left_on='lkey', right_on='rkey')

# you could also try this...
# tmp2.merge(tmp3) # if you don't specify the column, pandas will try to figure it out! 
# but this is dangerous - better to be explicit than not.

## Pt. 3: Join together and finish 'long to wide'

In [159]:
# here's the better way to do the merge
cleandata = tmp2.merge(tmp3, left_on='Town', right_on='Town',
                         how='outer', suffixes=('', '_y'))
cleandata.head()


,Town,FIPS,Year,Median Home Value,FIPS_y,Year_y,Ratio to State Median
0,Andover,901301080,2006-2010,276300.0,901301080,2006-2010,0.93
1,Ansonia,900901220,2006-2010,267300.0,900901220,2006-2010,0.90
2,Ashford,901501430,2006-2010,267900.0,901501430,2006-2010,0.90
3,Avon,900302060,2006-2010,421100.0,900302060,2006-2010,1.42
4,Barkhamsted,900502760,2006-2010,294900.0,900502760,2006-2010,0.99


In [160]:
cleandata.shape

(169, 7)

Take a look and see that we have a few duplicate columns (look at the subscripts with a _y). There are a few ways to deal with this...

* https://stackoverflow.com/questions/19125091/pandas-merge-how-to-avoid-duplicating-columns

One way is to drop columns with names that contain a subscript.

In [161]:
# drop columns with a _y
cleandata = cleandata[cleandata.columns.drop(list(cleandata.filter(regex='_y')))]
print(cleandata.shape)
cleandata.head()

(169, 5)


,Town,FIPS,Year,Median Home Value,Ratio to State Median
0,Andover,901301080,2006-2010,276300.0,0.93
1,Ansonia,900901220,2006-2010,267300.0,0.90
2,Ashford,901501430,2006-2010,267900.0,0.90
3,Avon,900302060,2006-2010,421100.0,1.42
4,Barkhamsted,900502760,2006-2010,294900.0,0.99


And to finish up, let's confirm our shape.

In [162]:
cleandata.shape # 169... perfection!

(169, 5)

You could easily extend this for each year, name columns after years, and then merge all 8 together. This may be nicer for modeling later on!

# On Your Own
Try to subset rows for 2006-2010 for the education data and join it to the housing data!

# (optional) Make a pivot table!
Read the documentation for a pivot table, then give it a try!
* https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [163]:
df_home

,Town,FIPS,Year,Measure Type,Variable,Value
0,Andover,901301080,2006-2010,Number,Median Home Value,276300.00
1,Andover,901301080,2006-2010,Number,Margins of Error,21727.00
2,Andover,901301080,2006-2010,Ratio to State Median,Median Home Value,0.93
3,Andover,901301080,2006-2010,Ratio to State Median,Margins of Error,0.07
4,Andover,901301080,2007-2011,Number,Median Home Value,303500.00
...,...,...,...,...,...,...
5435,Woodstock,901588190,2012-2016,Ratio to State Median,Margins of Error,0.06
5436,Woodstock,901588190,2013-2017,Number,Median Home Value,247500.00
5437,Woodstock,901588190,2013-2017,Number,Margins of Error,14279.00
5438,Woodstock,901588190,2013-2017,Ratio to State Median,Median Home Value,0.92


In [164]:
# notice that 'Variable' is a part of the index
table = pd.pivot_table(df_home, values='Value', index=['Town','Variable'],
                    columns=['Year'], aggfunc=np.mean)
table

Year                           2006-2010   2007-2011   2008-2012   2009-2013  \
Town       Variable                                                            
Andover    Margins of Error    10863.535   13718.545   14822.550   17755.065   
           Median Home Value  138150.465  151750.520  151250.530  143600.515   
Ansonia    Margins of Error     3826.515    4483.015    4586.015    4155.015   
           Median Home Value  133650.450  132250.450  126600.445  116450.420   
Ashford    Margins of Error     8757.030    9028.530    9113.530    7129.525   
...                                  ...         ...         ...         ...   
Woodbridge Median Home Value  248500.840  242450.825  243100.850  237050.850   
Woodbury   Margins of Error    11215.540   10138.535    9876.535    6870.025   
           Median Home Value  200000.675  194800.665  185500.650  180200.645   
Woodstock  Margins of Error     9270.030   10702.035   12248.045   19952.570   
           Median Home Value  142750.480  140200.480  136900.480  136150.490   

Year                           2010-2014   2011-2015   2012-2016   2013-2017  
Town       Variable                                                           
Andover    Margins of Error    12039.045   14479.055    7808.030    6886.525  
           Median Home Value  151600.550  138100.510  140850.525  138550.515  
Ansonia    Margins of Error     3157.010    3923.515    3939.515    5386.020  
           Median Home Value  109600.400  108250.400  107400.400  104350.385  
Ashford    Margins of Error     4845.520    7007.025    7697.030    9421.535  
...                                  ...         ...         ...         ...  
Woodbridge Median Home Value  234000.850  237250.875  242150.900  239300.885  
Woodbury   Margins of Error     9122.035   10714.540    8709.530    9456.035  
           Median Home Value  179400.655  167250.620  165900.615  175400.650  
Woodstock  Margins of Error    12679.045    6829.025    7538.530    7139.525  
           Median Home Value  123100.450  121250.450  123300.460  123750.460  

[340 rows x 8 columns]

In [165]:
table.reset_index(inplace=True)
table

Year,Town,Variable,2006-2010,2007-2011,2008-2012,2009-2013,2010-2014,2011-2015,2012-2016,2013-2017
0,Andover,Margins of Error,10863.535,13718.545,14822.550,17755.065,12039.045,14479.055,7808.030,6886.525
1,Andover,Median Home Value,138150.465,151750.520,151250.530,143600.515,151600.550,138100.510,140850.525,138550.515
2,Ansonia,Margins of Error,3826.515,4483.015,4586.015,4155.015,3157.010,3923.515,3939.515,5386.020
3,Ansonia,Median Home Value,133650.450,132250.450,126600.445,116450.420,109600.400,108250.400,107400.400,104350.385
4,Ashford,Margins of Error,8757.030,9028.530,9113.530,7129.525,4845.520,7007.025,7697.030,9421.535
...,...,...,...,...,...,...,...,...,...,...
335,Woodbridge,Median Home Value,248500.840,242450.825,243100.850,237050.850,234000.850,237250.875,242150.900,239300.885
336,Woodbury,Margins of Error,11215.540,10138.535,9876.535,6870.025,9122.035,10714.540,8709.530,9456.035
337,Woodbury,Median Home Value,200000.675,194800.665,185500.650,180200.645,179400.655,167250.620,165900.615,175400.650
338,Woodstock,Margins of Error,9270.030,10702.035,12248.045,19952.570,12679.045,6829.025,7538.530,7139.525


In [166]:
# notice that 'Variable' is a part of the columns
table = pd.pivot_table(df_home, values='Value', index=['Town'],
                    columns=['Year','Variable'], aggfunc=np.mean)
table

Year                 2006-2010                          2007-2011  \
Variable      Margins of Error Median Home Value Margins of Error   
Town                                                                
Andover              10863.535        138150.465        13718.545   
Ansonia               3826.515        133650.450         4483.015   
Ashford               8757.030        133950.450         9028.530   
Avon                 10076.535        210550.710        10094.535   
Barkhamsted           9119.030        147450.495        10910.035   
...                        ...               ...              ...   
Windsor Locks         3230.010        106750.360         2973.010   
Wolcott               4749.015        133200.450         6247.020   
Woodbridge           17024.055        248500.840        10702.035   
Woodbury             11215.540        200000.675        10138.535   
Woodstock             9270.030        142750.480        10702.035   

Year                                   2008-2012                    \
Variable      Median Home Value Margins of Error Median Home Value   
Town                                                                 
Andover              151750.520        14822.550        151250.530   
Ansonia              132250.450         4586.015        126600.445   
Ashford              133700.455         9113.530        128400.450   
Avon                 203250.695         8870.530        195450.685   
Barkhamsted          141550.485         7615.525        137250.480   
...                         ...              ...               ...   
Windsor Locks        106450.365         3764.015        104900.365   
Wolcott              131650.450         5675.520        125600.440   
Woodbridge           242450.825        12269.545        243100.850   
Woodbury             194800.665         9876.535        185500.650   
Woodstock            140200.480        12248.045        136900.480   

Year                 2009-2013                          2010-2014  \
Variable      Margins of Error Median Home Value Margins of Error   
Town                                                                
Andover              17755.065        143600.515        12039.045   
Ansonia               4155.015        116450.420         3157.010   
Ashford               7129.525        119000.425         4845.520   
Avon                  8129.030        189200.680         7697.530   
Barkhamsted           8468.030        128900.460        10374.040   
...                        ...               ...              ...   
Windsor Locks         4292.015        104250.375         5210.020   
Wolcott               5328.020        123700.445         4450.515   
Woodbridge           12473.045        237050.850        10662.040   
Woodbury              6870.025        180200.645         9122.035   
Woodstock            19952.570        136150.490        12679.045   

Year                                   2011-2015                    \
Variable      Median Home Value Margins of Error Median Home Value   
Town                                                                 
Andover              151600.550        14479.055        138100.510   
Ansonia              109600.400         3923.515        108250.400   
Ashford              116500.425         7007.025        116000.430   
Avon                 189500.690         7585.030        188250.695   
Barkhamsted          127900.465        10276.540        125750.465   
...                         ...              ...               ...   
Windsor Locks        101050.370         3522.015         97900.360   
Wolcott              121650.445         4877.520        120450.445   
Woodbridge           234000.850        10483.540        237250.875   
Woodbury             179400.655        10714.540        167250.620   
Woodstock            123100.450         6829.025        121250.450   

Year                 2012-2016                          2013-2017  \
Variable      Margins of Error Medi